<h1>Code 2 Cure 20_Mars Notebook</h1>
<p>
This notebook showcases our model developped near 11 hours ago wich gives us 0.8939 Public LB and 1 on Private</p>

In [4]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import models, transforms
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import timm
from torch.nn.utils import spectral_norm

# ======================
# Configuration
# ======================
class CFG:
    # System
    seed = 42
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    num_workers = 2
    
    # Data
    data_path = '/kaggle/input/code2cure/images/CAMAIRAI BOX READY'
    train_csv = '/kaggle/input/code2cure/Train.csv'
    test_csv = '/kaggle/input/code2cure/Test.csv'
    img_size = 224  # SonoNet typically uses 224x224
    batch_size = 32
    
    # Model
    model_name = 'sononet'
    pretrained = True
    num_classes = 1
    dropout = 0.7
    
    # Training
    freeze_epochs = 10  # Stage 1 duration
    head_lr = 1e-4      # Learning rate for head during Stage 1
    backbone_lr = 1e-5  # Learning rate for backbone during Stage 2
    num_folds = 5
    epochs = 30
    lr = 3e-5
    weight_decay = 1e-4
    early_stop_patience = 7
    tta_steps = 11

    # Class imbalance
    pos_weight = 2.0
    mixup_alpha = 0.4
    cutmix_alpha = 1.0

# Set reproducibility
def seed_everything(seed=CFG.seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything()

# ======================
# Dataset & Augmentation (Modified for Ultrasound)
# ======================
class UltrasoundDataset(Dataset):
    def __init__(self, df, transform=None, is_test=False):
        self.df = df
        self.transform = transform
        self.is_test = is_test
        self.image_dir = CFG.data_path
       
    def __len__(self):
        return len(self.df)
   
    def __getitem__(self, idx):
        img_id = self.df.iloc[idx]['img_IDs']
        img_path = os.path.join(self.image_dir, f"{img_id}.jpg")
       
        try:
            # Convert to grayscale for ultrasound
            image = Image.open(img_path).convert('L')  # Changed to grayscale
            image = np.array(image)
            
            # Add channel dimension
            image = np.expand_dims(image, axis=-1)
            
            if self.transform:
                image = self.transform(image=image)['image']
               
            if self.is_test:
                return image, img_id
            else:
                label = self.df.iloc[idx]['target']
                return image, torch.tensor(label, dtype=torch.float)
           
        except Exception as e:
            print(f"Error loading {img_path}: {str(e)}")
            dummy = torch.zeros(1, CFG.img_size, CFG.img_size)  # Single channel
            return dummy, torch.tensor(0.0)

def get_augmentations(train=True):
    if train:
        return A.Compose([
            A.Resize(CFG.img_size, CFG.img_size),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.ElasticTransform(alpha=1, sigma=50, p=0.3),
            A.GaussianBlur(blur_limit=(3, 7), p=0.3),
            A.RandomBrightnessContrast(
                brightness_limit=(-0.2, 0.2),
                contrast_limit=(-0.2, 0.2),
                p=0.5
            ),
            A.CoarseDropout(
                max_holes=8,  # Reduced for medical images
                max_height=16,
                max_width=16,
                fill_value=0,
                p=0.3
            ),
            A.Normalize(mean=[0.5], std=[0.5]),  # Single channel
            ToTensorV2()
        ])
    else:
        return A.Compose([
            A.Resize(CFG.img_size, CFG.img_size),
            A.Normalize(mean=[0.5], std=[0.5]),
            ToTensorV2()
        ])

# ======================
# Modified Model Architecture with SonoNet
# ======================

class MedicalImageClassifier(nn.Module):
    def __init__(self, num_classes=1, pretrained=True):
        super().__init__()
        
        # Load SonoNet base model
        self.sononet = timm.create_model(
            'tf_mobilenetv3_small_075',
            pretrained=pretrained,
            in_chans=1,  # Grayscale input
            features_only=True
        )
        
        # Get channel dimensions from the backbone
        with torch.no_grad():
            dummy_input = torch.randn(1, 1, CFG.img_size, CFG.img_size)
            features = self.sononet(dummy_input)
            in_channels = features[-1].shape[1]  # Dynamically get input channels
        
        # Ultrasound-specific modifications
        self.attention = nn.Sequential(
            nn.Conv2d(in_channels, 1, kernel_size=3, padding=1),
            nn.Sigmoid()
        )
        
        # Modified head
        self.head = nn.Sequential(
            nn.Linear(in_channels, 512),
            nn.BatchNorm1d(512),
            nn.SiLU(),
            nn.Dropout(CFG.dropout),
            spectral_norm(nn.Linear(512, num_classes)),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Extract features
        features = self.sononet(x)[-1]
        
        # Apply attention
        att = self.attention(features)
        features = features * att
        
        # Pooling and classification
        features = F.adaptive_avg_pool2d(features, (1, 1)).flatten(1)
        return self.head(features).squeeze()
# ======================
# Training Engine (No changes needed)
# ======================
def mixup_data(images, labels, alpha=1.0):
    if alpha <= 0:
        return images, labels
    
    batch_size = images.size(0)
    indices = torch.randperm(batch_size, device=images.device)
    shuffled_images = images[indices]
    shuffled_labels = labels[indices]
    
    lam = np.random.beta(alpha, alpha)
    mixed_images = lam * images + (1 - lam) * shuffled_images
    
    return mixed_images, lam * labels + (1 - lam) * shuffled_labels

def cutmix_data(images, labels, alpha=1.0):
    if alpha <= 0:
        return images, labels
    
    batch_size, _, H, W = images.shape
    indices = torch.randperm(batch_size, device=images.device)
    shuffled_labels = labels[indices]
    
    lam = np.random.beta(alpha, alpha)
    rx = np.random.uniform(0, W)
    ry = np.random.uniform(0, H)
    rw = W * np.sqrt(1 - lam)
    rh = H * np.sqrt(1 - lam)
    x1 = int(np.clip(rx - rw / 2, 0, W))
    y1 = int(np.clip(ry - rh / 2, 0, H))
    x2 = int(np.clip(rx + rw / 2, 0, W))
    y2 = int(np.clip(ry + rh / 2, 0, H))
    
    mixed_images = images.clone()
    mixed_images[:, :, y1:y2, x1:x2] = images[indices, :, y1:y2, x1:x2]
    
    lam = 1 - ((x2 - x1) * (y2 - y1) / (W * H))
    return mixed_images, lam * labels + (1 - lam) * shuffled_labels

# Update the train_epoch function
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    pbar = tqdm(loader, total=len(loader))
    
    for images, labels in pbar:
        # Move data to device first
        images = images.to(CFG.device)
        labels = labels.to(CFG.device)
        
        # Apply augmentations
        if np.random.rand() < 0.5:
            images, labels = mixup_data(images, labels, alpha=CFG.mixup_alpha)
        elif np.random.rand() < 0.5:
            images, labels = cutmix_data(images, labels, alpha=CFG.cutmix_alpha)
        
        # Label smoothing
        labels = labels * 0.9 + 0.05
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        pbar.set_postfix({'loss': loss.item()})
    
    return running_loss / len(loader.dataset)

def validate_epoch(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    preds, targets = [], []
   
    with torch.no_grad():
        for images, labels in tqdm(loader, total=len(loader)):
            images = images.to(CFG.device)
            labels = labels.to(CFG.device)
           
            outputs = model(images)
            loss = criterion(outputs, labels)
           
            running_loss += loss.item() * images.size(0)
            preds.extend(torch.sigmoid(outputs).cpu().numpy())
            targets.extend(labels.cpu().numpy())
   
    # Check if there's more than one class before calculating AUC
    unique_classes = np.unique(targets)
    if len(unique_classes) < 2:
        print("Warning: Only one class present in validation set. Skipping AUC calculation.")
        auc = 0.0
    else:
        auc = roc_auc_score(targets, preds)
    return running_loss / len(loader.dataset), auc


class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return F_loss.mean()

# Replace criterion with:
criterion = FocalLoss(alpha=CFG.pos_weight)


# ======================
# Modified K-Fold Training
# ======================
# ======================
# Modified K-Fold Training
# ======================
def run_kfold_training():
    df = pd.read_csv(os.path.join(CFG.data_path, CFG.train_csv))
    skf = RepeatedStratifiedKFold(
            n_splits=5,
            n_repeats=3,
            random_state=CFG.seed
        )
           
    fold_models = []
   
    for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['target'])):
        print(f"\n=== Fold {fold+1}/{CFG.num_folds} ===")
        
        # Data preparation
        train_df = df.iloc[train_idx]
        val_df = df.iloc[val_idx]
       
        # Handle class imbalance
        class_counts = train_df['target'].value_counts()
        weights = 1. / class_counts[train_df['target']].values
        sampler = WeightedRandomSampler(weights, len(weights)*2, replacement=True)
       
        # Create datasets
        train_ds = UltrasoundDataset(train_df, get_augmentations(True))
        val_ds = UltrasoundDataset(val_df, get_augmentations(False))
       
        train_loader = DataLoader(
            train_ds,
            batch_size=CFG.batch_size,
            sampler=sampler,
            num_workers=CFG.num_workers
        )
        val_loader = DataLoader(
            val_ds,
            batch_size=CFG.batch_size*2,
            shuffle=False,
            num_workers=CFG.num_workers
        )
       
        # Model setup
        model = MedicalImageClassifier().to(CFG.device)
        
        # Parameter groups
        backbone_params = model.sononet.parameters()
        head_params = list(model.attention.parameters()) + list(model.head.parameters())

        # Freeze backbone initially
        for param in backbone_params:
            param.requires_grad = False

        # Optimizer with separate parameter groups
        optimizer = optim.AdamW(
            [
                {'params': backbone_params, 'lr': 0},  # Inactive during Stage 1
                {'params': head_params, 'lr': CFG.head_lr}
            ],
            weight_decay=CFG.weight_decay
        )
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)

        best_auc = 0
        no_improve = 0
        best_weights = None

        # Training loop
        for epoch in range(CFG.epochs):
            # Stage transition
            if epoch == CFG.freeze_epochs:
                print("\n=== Unfreezing backbone ===")
                # Unfreeze backbone
                for param in backbone_params:
                    param.requires_grad = True
                # Update optimizer parameters
                optimizer.param_groups[0]['lr'] = CFG.backbone_lr

            train_loss = train_epoch(model, train_loader, optimizer, criterion)
            val_loss, val_auc = validate_epoch(model, val_loader, criterion)
            scheduler.step(val_auc)
           
            print(f"Epoch {epoch+1}/{CFG.epochs}")
            print(f"Train Loss: {train_loss:.4f}")
            print(f"Val Loss: {val_loss:.4f} | Val AUC: {val_auc:.4f}")
           
            # Early stopping
            if val_auc > best_auc:
                best_auc = val_auc
                no_improve = 0
                best_weights = model.state_dict().copy()
                torch.save(model.state_dict(), f"fold{fold}_best.pth")
            else:
                no_improve += 1
                if no_improve >= CFG.early_stop_patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break
       
        if best_weights is not None:
            model.load_state_dict(best_weights)
        fold_models.append(model)
   
    return fold_models

# ======================
# Inference & Ensemble (No changes needed)
# ======================
def predict_tta(model, loader):
    preds = []
    ids = []
   
    with torch.no_grad():
        for images, img_ids in tqdm(loader):
            images = images.to(CFG.device)
            batch_preds = []
           
            for _ in range(CFG.tta_steps):
                outputs = model(images)
                batch_preds.append(torch.sigmoid(outputs).cpu().numpy())
           
            avg_preds = np.mean(batch_preds, axis=0)
            preds.extend(avg_preds)
            ids.extend(img_ids)
   
    return ids, preds

# Main execution
if __name__ == '__main__':
    # Train models
    models = run_kfold_training()
    
    # Load test data
    test_df = pd.read_csv(os.path.join(CFG.data_path, CFG.test_csv))
    test_ds = UltrasoundDataset(test_df, get_augmentations(False), is_test=True)
    test_loader = DataLoader(
        test_ds,
        batch_size=CFG.batch_size*2,
        shuffle=False,
        num_workers=CFG.num_workers
    )
    
    # Ensemble predictions
    all_preds = []
    for model in models:
        ids, preds = predict_tta(model, test_loader)
        all_preds.append(preds)
    
    final_preds = np.mean(all_preds, axis=0)
    submission = pd.DataFrame({
        'img_IDs': ids,
        'target': final_preds
    })
    submission.to_csv('/kaggle/working/submission.csv', index=False)


=== Fold 1/5 ===


100%|██████████| 1/1 [00:00<00:00,  3.97it/s]


Epoch 1/30
Train Loss: 1.5189
Val Loss: 0.3688 | Val AUC: 0.7188


100%|██████████| 1/1 [00:00<00:00,  4.02it/s]


Epoch 2/30
Train Loss: 1.1832
Val Loss: 0.3377 | Val AUC: 0.7396


...

100%|██████████| 1/1 [00:00<00:00,  4.48it/s]


Epoch 19/30
Train Loss: 0.9918
Val Loss: 0.3095 | Val AUC: 0.7582


100%|██████████| 1/1 [00:00<00:00,  4.41it/s]


Epoch 20/30
Train Loss: 0.9089
Val Loss: 0.3110 | Val AUC: 0.7582
Early stopping at epoch 20


100%|██████████| 1/1 [00:00<00:00,  2.69it/s]
